 #### Project 3 - Part 2
 
 - Paula Pipkin

In [1]:
import pandas as pd
import os, time,json
import tmdbsimple as tmdb 
from tqdm.notebook import tqdm_notebook

In [2]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [3]:
def get_info(movie_id, data):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    movie_info = movie.info()
    fetch = {'imdb_id':movie_id, 'info':movie_info[data]}
            
    return fetch

In [4]:
#get credentials

with open('/Users/paula/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
tmdb.API_KEY =  login['api-key']

### Part 2: Use an API to extract box office revenue and profit data to add to your IMDB data and perform exploratory data analysis.

- Checking How ID appears in our table

In [5]:
df_basics = pd.read_csv('data/title_basics.csv.gz')
df_basics.columns.values

array(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'runtimeMinutes', 'genres'], dtype=object)

In [6]:
df_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,126,Drama


In [7]:
ids = df_basics['tconst'].values
partial = ids[0:300]
partial

array(['tt0035423', 'tt0062336', 'tt0069049', 'tt0088751', 'tt0096056',
       'tt0100275', 'tt0108549', 'tt0113026', 'tt0113092', 'tt0114447',
       'tt0115937', 'tt0116391', 'tt0116628', 'tt0116991', 'tt0117743',
       'tt0118141', 'tt0118589', 'tt0118652', 'tt0118694', 'tt0118710',
       'tt0118852', 'tt0118926', 'tt0119004', 'tt0119231', 'tt0119273',
       'tt0119495', 'tt0119806', 'tt0119966', 'tt0119970', 'tt0119980',
       'tt0120166', 'tt0120202', 'tt0120263', 'tt0120467', 'tt0120589',
       'tt0120607', 'tt0120624', 'tt0120626', 'tt0120630', 'tt0120667',
       'tt0120679', 'tt0120681', 'tt0120698', 'tt0120733', 'tt0120737',
       'tt0120753', 'tt0120755', 'tt0120804', 'tt0120807', 'tt0120824',
       'tt0120903', 'tt0120912', 'tt0120913', 'tt0120917', 'tt0121164',
       'tt0121765', 'tt0121766', 'tt0122161', 'tt0122247', 'tt0122459',
       'tt0123003', 'tt0123055', 'tt0123581', 'tt0124889', 'tt0125022',
       'tt0126029', 'tt0126899', 'tt0127349', 'tt0128158', 'tt01

In [ ]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

info_TO_GET = ['budget','revenue']
errors = [ ]

# Start of OUTER loop
for info in tqdm_notebook(info_TO_GET, desc='info', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{info}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0,'info':0 }],f)
    # saving movie ids to list
    movie_ids = df_basics['tconst'].copy()
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
        #Get index and movie id from list # partial
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies {info}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_info(movie_id,info)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
    final_info_df = pd.read_json(JSON_FILE)
    final_info_df.to_csv(f"{FOLDER}final_tmdb_data_{info}.csv.gz", compression="gzip", index=False)

print(f"- Total errors: {len(errors)}")

info:   0%|          | 0/2 [00:00<?, ?it/s]

Movies budget:   0%|          | 0/82476 [00:00<?, ?it/s]

In [ ]:
df_budget = pd.read_csv('data/final_tmdb_data_budget.csv.gz')
df_budget.head()